# LSTM

## CalamanCy

Calamancy vectors are trained from a skip-gram model.

In [1]:
import numpy as np
import pandas as pd
import string
from tqdm import tqdm
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt

/tmp/ipykernel_37046/3484305165.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Read Data

In [7]:
with open('./misc/stopwords.txt') as f:
  stopwords = f.read().replace('\n', ' ').split()

In [3]:
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 'her',
 'hers',
 'herself',
 'it',
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 '

In [8]:
with open('./misc/training_text.txt') as f:
  text = f.read().replace('\n', '')
  text = text.translate(
    str.maketrans('', '', string.punctuation)
  )
  text = ''.join([t for t in text if t not in '0123456789'])
  text = text.lower().split()

text = [w for w in text if w not in stopwords][:2000]

In [9]:
text

['today',
 'learning',
 'fundamentals',
 'data',
 'science',
 'statistics',
 'data',
 'science',
 'statistics',
 'hot',
 'growing',
 'fields',
 'alternative',
 'names',
 'machine',
 'learning',
 'artificial',
 'intelligence',
 'big',
 'data',
 'etc',
 'im',
 'really',
 'excited',
 'talk',
 'data',
 'science',
 'statistics',
 'data',
 'science',
 'statistics',
 'long',
 'passions',
 'mine',
 'didnt',
 'used',
 'good',
 'data',
 'science',
 'statistics',
 'studying',
 'data',
 'science',
 'statistics',
 'long',
 'time',
 'got',
 'better',
 'better',
 'became',
 'data',
 'science',
 'statistics',
 'expert',
 'im',
 'really',
 'excited',
 'talk',
 'data',
 'science',
 'statistics',
 'thanks',
 'listening',
 'talk',
 'data',
 'science',
 'statistics']

### Prepare Training Data

In [16]:
# Introduce negative samples so model knows what are
# context words as well as what are not context words

WINDOW_SIZE = 3
NUM_NEGATIVE_SAMPLES = 3

data = []

# Iterate over all words
for index, center_word in enumerate(
  text[WINDOW_SIZE - 1 : -WINDOW_SIZE]
):
  # Iterate over context words around center word
  context_words = [
    context_word 
    for context_word 
    in text[index : index + 2 * WINDOW_SIZE-1] 
    if context_word != center_word
  ]

  # Get words not in context as negative samples
  for context_word in context_words:
    data.append([center_word, context_word, 1])
    negative_samples = np.random.choice([
      w 
      for w
      in text[WINDOW_SIZE - 1 : -WINDOW_SIZE]
      if w != center_word
      and w not in context_words
    ], NUM_NEGATIVE_SAMPLES)

    for negative_sample in negative_samples:
      data.append([center_word, negative_sample, 0])

In [17]:
pd.DataFrame(data)

,0,1,2
0,fundamentals,today,1
1,fundamentals,long,0
2,fundamentals,growing,0
3,fundamentals,etc,0
4,fundamentals,learning,1
...,...,...,...
979,talk,better,0
980,talk,science,1
981,talk,got,0
982,talk,im,0


## LSTM Architecture

![Alt text](assets/image.png)

### Forget Gate

![Alt text](assets/image-4.png)

$ f_t = sigmoid(W_f * [h_{t-1},x_t] + b_f) $

Forget gate decides what information will be retained in the cell state.

Multiply the weight of the forget gate $ W_t $ with previous state $ h_{t-1} $ and current input $ x_t $. Add the bias of the forget gate $ b_f $.

The sigmoid function turns this into a value between 0 and 1 indicating how much percent of cell state to keep.

```python

```

### Input Gate

![Alt text](assets/image-5.png)

$ i_t = sigmoid(W_i * [h_{t-1},x_t] + b_i) $

The input gate decides which values to update in the cell state and by how much.

$ \~{C_t} = tanh(W_C * [h_{t-1},x_t] + b_C) $

$ \~{C_t} $ determines the new values to be added to the cell state.

### Cell State

![Alt text](assets/image-1.png)

$ C_t = f_t * C_{t-1} + i_t * \tilde{C} $

Cell state is the long term memory. It forgets some information from the previous step in the sequence, then adds new information calculated from the input gate, forming the updated cell state.

### Output Gate

![Alt text](./assets/image-6.png)

$ o_t = sigmoid(W_o [h_{t-1},x_t] + b_o) $

The output gate determines which values in the cell state will be forwarded.

$ h_t = o_t * tanh(C_t) $

The hidden state calculates the output by multiplying the output gate and a $ tanh $-filtered cell state. It doubly functions as the short term memory for the next input in the sequence.

The `hidden_size` parameter determines the size, or dimensions, of the hidden state.

### Source

https://github.com/pytorch/pytorch/blob/9347a79f1c35c76535310111d1c50bada4e975a8/aten/src/ATen/native/RNN.cpp#L716

Get result of $ W_h(h_{t-1}) + W_i(x_t) + b_h + b_i $
```cpp
const auto gates = params.linear_hh(hx).add_(
    pre_compute_input ? input : params.linear_ih(input));
```

Apply activation to each gate
```cpp
auto chunked_gates = gates.unsafe_chunk(4, 1);
auto ingate = chunked_gates[0].sigmoid_();
auto forgetgate = chunked_gates[1].sigmoid_();
auto cellgate = chunked_gates[2].tanh_();
auto outgate = chunked_gates[3].sigmoid_();
```

Update cell state (long term) and hidden state (short term)
```cpp
auto cy = (forgetgate * cx).add_(ingate * cellgate);
auto hy = outgate * cy.tanh();
return std::make_tuple(std::move(hy), std::move(cy));
```